In [ ]:
!pip install -U spacy==2.1.0
!python -m spacy download en

!pip install Cython --install-option="–no-cython-compile"

!pip install neuralcoref

!pip install benepar

!python -m spacy download en_core_web_md

!spacy download en

In [15]:
import nltk
nltk.download('punkt')

import pandas as pd

import spacy
import benepar
import neuralcoref

from nltk.tokenize import sent_tokenize

class MicroStatments:
  """
  return microstatemnts from input mwp
  coref resolved and conjunction resolved.
  """
  
  def __init__(self):
    self.nlp_pronoun = spacy.load('en')
    neuralcoref.add_to_pipe(self.nlp_pronoun, greedyness=0.52)
    benepar.download('benepar_en3')
    self.nlp = spacy.load('en_core_web_md')
    if spacy.__version__.startswith('2'):
        self.nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
    else:
        self.nlp.add_pipe("benepar", config={"model": "benepar_en3"})

  def __replace(self, sentence):
    doc = self.nlp_pronoun(sentence)
    return doc._.coref_resolved

  def __extract_sub_verb_object(self, sentence):
    subject = ""
    verb_phrase = ""
    preposition_phrase = ""
    #dependency_tree = sent1._.parse_string
    doc = self.nlp(sentence)
    sent = list(doc.sents)[0]
    dependency_tree = sent._.parse_string
    sb = 1
    vb = 0
    pb = 0

    for i in range(len(dependency_tree)):
      ch = dependency_tree[i]

      if sb == 1:
        if (ch.islower()==True or ch.isdigit()==True or ch==" " or ch=="," ):
          if( (subject[:-1]!=" " and ch==" ") or ch!=" "):
              subject = subject+ch
        elif ch=="V":
          sb = 0
          vb = 1
          continue

      if vb == 1:
        if (ch.islower()==True or ch.isdigit()==True or ch==" " or ch==","):
          if( (verb_phrase[:-1]!=" " and ch==" ") or ch!=" "):
            verb_phrase = verb_phrase+ch
          
        elif ch=="N" and dependency_tree[i+1] == "P":
          vb = 0
          pb = 1
          continue

      if pb == 1:
          if (ch.islower()==True or ch.isdigit()==True or ch==" " or ch==","):
            if( (preposition_phrase[:-1]!=" " and ch==" ") or ch!=" "):
              preposition_phrase = preposition_phrase+ch
      
    return {"subject":subject.strip(), "verb":verb_phrase.strip(), "object":preposition_phrase.strip()}


  def __split_conj(self, sent_list):
    sub = ""
    l = list()

    for i in sent_list:
      if i!="and":
          sub = sub+" "+i
      else:
        if sub!="":
          l.append(sub)
          sub = ""

    l.append(sub)
    return l
  
  def __handle_conjunction(self, sentence):
    d = self.__extract_sub_verb_object(sentence)

    subject = d["subject"].replace(",","and")
    verb = d["verb"].replace(",","and")
    obj = d["object"].replace(",","and")
    subject_tokens = [i for i in subject.split(' ') if i != ' ']
    verb_tokens = [i for i in verb.split(' ') if i!= ' ']
    obj_tokens = [i for i in obj.split(' ') if i!=''] 
    sentences = list()
    if "and" in subject_tokens:
      split_subject = self.__split_conj(subject_tokens)
      
      for i in split_subject:
        sentences.append(i+" "+verb+" "+obj)
      return sentences
    if "and" in obj_tokens:
      
      split_obj = self.__split_conj(obj_tokens)
    
      
      for i in split_obj:
        sentences.append(subject+" " + verb+" "+ i)
        
      return sentences
    else:
      sentences.append(subject+" "+verb+" "+obj)
      return sentences
    
  def mwp_split(self, mwp):
    mwp_split_temp = sent_tokenize(mwp)
    mwp_split = list()
    res = []
    for i in mwp_split_temp:
      temp = self.__handle_conjunction(i)
      for j in temp:
        mwp_split.append(j.strip())
    for i, sent in enumerate(mwp_split):
      if "and" in sent:
        res.extend(self.__handle_conjunction(sent))
        continue
      res.append(sent)
    return res


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
ms = MicroStatments()

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [19]:
ms.mwp_split("john and rebecca had 2 apples and 2 oranges")

['john had  2 apples',
 'john had  2 oranges',
 'rebecca had  2 apples',
 'rebecca had  2 oranges']